<a href="https://colab.research.google.com/github/Gustavo-Ros/Simulaci-n-2/blob/main/Venta_de_boletos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install simpy

In [3]:

import matplotlib.pyplot as plt
import numpy as np
from random import random, seed
import pandas as pd
import simpy

In [4]:
def venta_boletos(env, fila_virtual, num_clientes, texto=True):
    global tasa_llegada, cifras, duracion_compra, tiempos_llegada, tiempos_compra, tiempos_salida

    for i in range(num_clientes):
        # Llegada del cliente a la fila virtual
        r = random()
        tiempo_llegada = -np.log(r) / tasa_llegada
        yield env.timeout(tiempo_llegada)
        tiempos_llegada.append(env.now)

        if texto:
            print(f'----El cliente número {i} ha llegado a la fila virtual en el minuto {round(env.now, cifras)}----')

        # Iniciar el proceso de compra para el cliente
        env.process(compra_boletos(env, fila_virtual, i, texto))

def compra_boletos(env, fila_virtual, num_cliente, texto=True):
    global duracion_compra, tiempos_compra, tiempos_salida, tiempo_compra_min, tiempo_compra_max

    # Solicitud para iniciar la compra de boletos
    with fila_virtual.request() as solicitud:
        yield solicitud

        # Se genera el tiempo de compra
        r = random()
        tiempo_compra = tiempo_compra_min + (tiempo_compra_max - tiempo_compra_min) * r
        duracion_compra += tiempo_compra
        tiempos_compra.append(tiempo_compra)

        if texto:
            print(f'****El cliente {num_cliente} ha comenzado la compra en el minuto {round(env.now, cifras)}****')

        # Tiempo en el sistema durante la compra de boletos
        yield env.timeout(tiempo_compra)

        # Registro del momento en que termina la compra
        tiempos_salida.append(env.now)

        if texto:
            print(f'xxxEl cliente {num_cliente} ha terminado la compra en el minuto {round(env.now, cifras)}xxx')

# Variables globales para la simulación
global tiempo_compra_min, tiempo_compra_max, tasa_llegada, num_boletos, cifras
semilla = 13
seed(semilla)

# Configuración de la simulación
num_boletos = 1
tiempo_compra_min = 1      # Tiempo mínimo de compra en minutos
tiempo_compra_max = 5      # Tiempo máximo de compra en minutos
tasa_llegada = 1 / 10      # Clientes llegan cada 10 minutos en promedio
num_clientes = 5           # Número de clientes en la simulación
cifras = 2                 # Número de decimales para los tiempos

# Variables para el seguimiento de tiempos
global duracion_compra, tiempos_llegada, tiempos_compra, tiempos_salida
duracion_compra = 0
tiempos_llegada = []
tiempos_compra = []
tiempos_salida = []

# Crear entorno de simulación y recurso (fila virtual)
env = simpy.Environment()
fila_virtual = simpy.Resource(env, capacity=num_boletos)

# Iniciar el proceso de venta de boletos
env.process(venta_boletos(env, fila_virtual, num_clientes))

# Ejecutar la simulación
env.run()


----El cliente número 0 ha llegado a la fila virtual en el minuto 13.51----
****El cliente 0 ha comenzado la compra en el minuto 13.51****
xxxEl cliente 0 ha terminado la compra en el minuto 17.25xxx
----El cliente número 1 ha llegado a la fila virtual en el minuto 17.29----
****El cliente 1 ha comenzado la compra en el minuto 17.29****
----El cliente número 2 ha llegado a la fila virtual en el minuto 18.92----
xxxEl cliente 1 ha terminado la compra en el minuto 19.03xxx
****El cliente 2 ha comenzado la compra en el minuto 19.03****
xxxEl cliente 2 ha terminado la compra en el minuto 20.62xxx
----El cliente número 3 ha llegado a la fila virtual en el minuto 33.59----
****El cliente 3 ha comenzado la compra en el minuto 33.59****
xxxEl cliente 3 ha terminado la compra en el minuto 37.53xxx
----El cliente número 4 ha llegado a la fila virtual en el minuto 48.5----
****El cliente 4 ha comenzado la compra en el minuto 48.5****
xxxEl cliente 4 ha terminado la compra en el minuto 50.02xxx
